Выполнил 
студент группы 309Б 
Мозговой Никита 

In [ ]:
!pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip


In [362]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport
import sklearn
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error, r2_score, make_scorer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression

In [188]:
df = pd.read_csv('train.csv')
df.head()

,ApplicationDate,Age,AnnualIncome,CreditScore,LoanAmount,LoanDuration,MaritalStatus,NumberOfDependents,HomeOwnershipStatus,MonthlyDebtPayments,...,EmploymentStatus,EducationLevel,Experience,NetWorth,BaseInterestRate,InterestRate,MonthlyLoanPayment,TotalDebtToIncomeRatio,LoanApproved,RiskScore
0,2010-06-26,27.0,66829.0,549.0,17290.0,60.0,Divorced,1.0,Rent,1095.0,...,Employed,Associate,4.0,35067.0,0.257790,0.251465,508.970230,0.288013,0.0,66.176500
1,1996-09-23,55.0,172147.0,850.0,16110.0,36.0,Widowed,1.0,Mortgage,211.0,...,Employed,High School,33.0,27001.0,0.086110,0.093173,514.675859,0.050585,1.0,28.495737
2,2015-01-19,51.0,300000.0,850.0,38436.0,36.0,Married,0.0,Mortgage,546.0,...,Employed,Bachelor,28.0,278382.0,0.108436,0.115443,1268.276385,0.072571,1.0,34.488104
3,1981-05-12,25.0,34683.0,847.0,19186.0,48.0,Married,0.0,Other,153.0,...,Employed,High School,0.0,9224.0,0.100686,0.112822,498.505187,0.225415,1.0,36.910753
4,1995-05-07,55.0,300000.0,850.0,30437.0,48.0,Single,2.0,Rent,562.0,...,Employed,Bachelor,31.0,4502.0,0.110437,0.089037,756.035156,0.052721,1.0,31.347091


In [190]:
df.shape

(11017, 36)

Удалим дубликаты и посмотрим как изменилась таблица

In [193]:
new_df = df.drop_duplicates()

In [195]:
new_df.shape

(10001, 36)

Проверим сколько пропусков у признаков

In [198]:
new_df.isnull().mean() * 100

ApplicationDate               0.009999
Age                           0.009999
AnnualIncome                  0.009999
CreditScore                   4.719528
LoanAmount                    4.719528
LoanDuration                  0.009999
MaritalStatus                 0.009999
NumberOfDependents            0.009999
HomeOwnershipStatus           0.009999
MonthlyDebtPayments           4.719528
CreditCardUtilizationRate     0.009999
NumberOfOpenCreditLines       0.009999
NumberOfCreditInquiries       0.009999
DebtToIncomeRatio             0.009999
BankruptcyHistory             4.719528
LoanPurpose                   4.719528
PreviousLoanDefaults          0.009999
PaymentHistory                0.009999
LengthOfCreditHistory         0.009999
SavingsAccountBalance         0.009999
CheckingAccountBalance        4.719528
TotalAssets                   4.719528
TotalLiabilities              0.009999
MonthlyIncome                 0.009999
UtilityBillsPaymentHistory    0.009999
JobTenure                

Ни у какого из признаков количество пропусков не привысило 70%, поэтому ничего удалять не будем. Тогда заполним пропуски категориальных признаков для начала

In [ ]:
imputer = SimpleImputer(missing_values = np.nan, strategy = 'most_frequent')
categorical_columns = ['MaritalStatus', 'HomeOwnershipStatus', 'LoanPurpose', 'EmploymentStatus', 'EducationLevel']
new_df[categorical_columns] = imputer.fit_transform(new_df[categorical_columns].values)

In [243]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8878 entries, 0 to 9999
Data columns (total 36 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   ApplicationDate             8878 non-null   object 
 1   Age                         8878 non-null   float64
 2   AnnualIncome                8878 non-null   float64
 3   CreditScore                 8878 non-null   float64
 4   LoanAmount                  8878 non-null   float64
 5   LoanDuration                8878 non-null   float64
 6   MaritalStatus               8878 non-null   object 
 7   NumberOfDependents          8878 non-null   float64
 8   HomeOwnershipStatus         8878 non-null   object 
 9   MonthlyDebtPayments         8878 non-null   float64
 10  CreditCardUtilizationRate   8878 non-null   float64
 11  NumberOfOpenCreditLines     8878 non-null   float64
 12  NumberOfCreditInquiries     8878 non-null   float64
 13  DebtToIncomeRatio           8878 non-n

Аналогично поступим с числовыми значениями

In [ ]:
imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean')
numeric_columns = ['CreditScore', 'LoanAmount', 'MonthlyDebtPayments', 'CheckingAccountBalance', 'TotalAssets', 'NetWorth', 'BaseInterestRate', 'BankruptcyHistory']
new_df[numeric_columns] = imputer.fit_transform(new_df[numeric_columns].values)

In [247]:
new_df.isnull().mean() * 100

ApplicationDate               0.0
Age                           0.0
AnnualIncome                  0.0
CreditScore                   0.0
LoanAmount                    0.0
LoanDuration                  0.0
MaritalStatus                 0.0
NumberOfDependents            0.0
HomeOwnershipStatus           0.0
MonthlyDebtPayments           0.0
CreditCardUtilizationRate     0.0
NumberOfOpenCreditLines       0.0
NumberOfCreditInquiries       0.0
DebtToIncomeRatio             0.0
BankruptcyHistory             0.0
LoanPurpose                   0.0
PreviousLoanDefaults          0.0
PaymentHistory                0.0
LengthOfCreditHistory         0.0
SavingsAccountBalance         0.0
CheckingAccountBalance        0.0
TotalAssets                   0.0
TotalLiabilities              0.0
MonthlyIncome                 0.0
UtilityBillsPaymentHistory    0.0
JobTenure                     0.0
EmploymentStatus              0.0
EducationLevel                0.0
Experience                    0.0
NetWorth      

In [249]:
new_df = new_df.dropna() #остоавшиеся нули(строки с ними) удалим и посмотрим сколько их было 

Получается была одна нулевая строка. Теперь попробуем обработать выбросы. Оставим только те данные которые укладываются в 99% доверительный интервал(посмотрим на примере возраста)

In [253]:
new_df.boxplot(column=['Age'], figsize = (2, 7))

<Axes: >

In [255]:
new_df.boxplot(column=['CreditScore'], figsize = (2, 7))

<Axes: >

In [257]:
new_df.boxplot(column=['RiskScore'], figsize = (2, 7))

<Axes: >

In [259]:
new_df = new_df[new_df['RiskScore'] < new_df['RiskScore'].quantile(0.99)] # and new_df['RiskScore'] > new_df['RiskScore'].quantile(0.01)]
new_df = new_df[new_df['RiskScore'] > new_df['RiskScore'].quantile(0.01)]

new_df = new_df[new_df['Age'] < new_df['Age'].quantile(0.99)]
new_df = new_df[new_df['Age'] > new_df['Age'].quantile(0.01)]


In [261]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8431 entries, 0 to 9999
Data columns (total 36 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   ApplicationDate             8431 non-null   object 
 1   Age                         8431 non-null   float64
 2   AnnualIncome                8431 non-null   float64
 3   CreditScore                 8431 non-null   float64
 4   LoanAmount                  8431 non-null   float64
 5   LoanDuration                8431 non-null   float64
 6   MaritalStatus               8431 non-null   object 
 7   NumberOfDependents          8431 non-null   float64
 8   HomeOwnershipStatus         8431 non-null   object 
 9   MonthlyDebtPayments         8431 non-null   float64
 10  CreditCardUtilizationRate   8431 non-null   float64
 11  NumberOfOpenCreditLines     8431 non-null   float64
 12  NumberOfCreditInquiries     8431 non-null   float64
 13  DebtToIncomeRatio           8431 non-n

Взглянем на профайл нашего датасета (чуть позже он появится тут )

Признаки с большим показателем кореляции можем удалить, ведь они не дадут нам принципиально новых заний, а только усложнят вычисления

In [352]:
new_df = new_df.drop('Experience', axis=1)
new_df = new_df.drop('InterestRate', axis=1)
new_df = new_df.drop('MonthlyIncome', axis=1)

In [263]:
new_df = new_df.drop(['MonthlyLoanPayment', 'BaseInterestRate'] , axis=1)

Для числовых признаков применим стандартизацию а для категориальных кодирование

In [287]:
new_df['MaritalStatus'] = new_df['MaritalStatus'].astype('category').cat.codes
new_df['HomeOwnershipStatus'] = new_df['HomeOwnershipStatus'].astype('category').cat.codes
new_df['LoanPurpose'] = new_df['LoanPurpose'].astype('category').cat.codes
new_df['EmploymentStatus'] = new_df['EmploymentStatus'].astype('category').cat.codes
new_df['EducationLevel'] = new_df['EducationLevel'].astype('category').cat.codes

In [291]:
new_df.columns

Index(['ApplicationDate', 'Age', 'AnnualIncome', 'CreditScore', 'LoanAmount',
       'LoanDuration', 'MaritalStatus', 'NumberOfDependents',
       'HomeOwnershipStatus', 'MonthlyDebtPayments',
       'CreditCardUtilizationRate', 'NumberOfOpenCreditLines',
       'NumberOfCreditInquiries', 'DebtToIncomeRatio', 'BankruptcyHistory',
       'LoanPurpose', 'PreviousLoanDefaults', 'PaymentHistory',
       'LengthOfCreditHistory', 'SavingsAccountBalance',
       'CheckingAccountBalance', 'TotalAssets', 'TotalLiabilities',
       'MonthlyIncome', 'UtilityBillsPaymentHistory', 'JobTenure',
       'EmploymentStatus', 'EducationLevel', 'Experience', 'NetWorth',
       'InterestRate', 'TotalDebtToIncomeRatio', 'LoanApproved', 'RiskScore'],
      dtype='object')

In [293]:
features = ['Age', 'AnnualIncome', 'CreditScore', 'LoanAmount',
       'LoanDuration', 'MaritalStatus', 'NumberOfDependents',
       'HomeOwnershipStatus', 'MonthlyDebtPayments',
       'CreditCardUtilizationRate', 'NumberOfOpenCreditLines',
       'NumberOfCreditInquiries', 'DebtToIncomeRatio', 'BankruptcyHistory',
       'LoanPurpose', 'PreviousLoanDefaults', 'PaymentHistory',
       'LengthOfCreditHistory', 'SavingsAccountBalance',
       'CheckingAccountBalance', 'TotalAssets', 'TotalLiabilities',
       'MonthlyIncome', 'UtilityBillsPaymentHistory', 'JobTenure',
       'EmploymentStatus', 'EducationLevel', 'Experience', 'NetWorth',
       'InterestRate', 'TotalDebtToIncomeRatio', 'LoanApproved', 'RiskScore']
x = new_df.loc[:, features].values

minMaxScaler = MinMaxScaler()
xScaled = minMaxScaler.fit_transform(x)
new_df_norm = pd.DataFrame(xScaled)

In [295]:
new_df_norm.head()

,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,31,32
0,0.142857,0.070732,0.452727,0.038827,0.444444,0.000000,0.166667,1.000000,0.099577,0.162179,...,0.882185,0.235294,0.0,0.00,0.088889,0.003025,0.260327,0.011566,0.0,0.727345
1,0.809524,0.214461,1.000000,0.036003,0.222222,1.000000,0.166667,0.000000,0.018222,0.188223,...,0.731368,0.117647,0.0,0.75,0.733333,0.002309,0.059214,0.001826,1.0,0.087628
2,0.714286,0.388944,1.000000,0.089423,0.222222,0.333333,0.000000,0.000000,0.049052,0.483642,...,0.892594,0.176471,0.0,0.25,0.622222,0.024634,0.087509,0.002728,1.0,0.189362
3,0.095238,0.026862,0.994545,0.043363,0.333333,0.333333,0.000000,0.333333,0.012884,0.202240,...,0.838861,0.176471,0.0,0.75,0.000000,0.000730,0.084178,0.008998,1.0,0.230492
4,0.809524,0.388944,1.000000,0.070284,0.333333,0.666667,0.333333,1.000000,0.050525,0.295596,...,0.855543,0.294118,0.0,0.25,0.688889,0.000311,0.053960,0.001914,1.0,0.136036


In [297]:
x = StandardScaler().fit_transform(x)
new_df_stand = pd.DataFrame(data = x, columns = features)
new_df_stand.head()

,Age,AnnualIncome,CreditScore,LoanAmount,LoanDuration,MaritalStatus,NumberOfDependents,HomeOwnershipStatus,MonthlyDebtPayments,CreditCardUtilizationRate,...,UtilityBillsPaymentHistory,JobTenure,EmploymentStatus,EducationLevel,Experience,NetWorth,InterestRate,TotalDebtToIncomeRatio,LoanApproved,RiskScore
0,-1.353438,-0.564141,-0.749403,-0.460373,0.277688,-1.632826,-0.399230,1.231509,1.096919,-0.834020,...,1.004125,-0.432463,-0.369195,-1.398757,-1.392369,-0.253141,0.550436,-0.258524,-1.034886,1.080940
1,1.468756,0.345835,0.981485,-0.504166,-0.705178,2.245750,-0.399230,-1.111193,-0.670480,-0.685522,...,0.105012,-1.336305,-0.369195,0.864759,1.493584,-0.271457,-1.112432,-0.542470,0.966290,-1.171514
2,1.065585,1.450520,0.981485,0.324402,-0.705178,-0.339968,-1.103542,-1.111193,-0.000708,0.998896,...,1.066174,-0.884384,-0.369195,-0.644252,0.996006,0.299372,-0.878478,-0.516177,0.966290,-0.813306
3,-1.555023,-0.841891,0.964233,-0.390008,-0.213745,-0.339968,-1.103542,-0.330292,-0.786441,-0.605601,...,0.745840,-0.884384,-0.369195,0.864759,-1.790431,-0.311824,-0.906020,-0.333387,0.966290,-0.668487
4,1.468756,1.450520,0.981485,0.027542,-0.213745,0.952891,0.305082,1.231509,0.031281,-0.073305,...,0.845291,0.019458,-0.369195,-0.644252,1.294553,-0.322547,-1.155877,-0.539915,0.966290,-1.001067


In [366]:
x = new_df_stand.loc[:, new_df_stand.columns != 'RiskScore']
y = new_df['RiskScore']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

model = Ridge()

model.fit(x_train, y_train)

y_train_pred = model.predict(x_train)
y_test_pred = model.predict(x_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MSE train: {mean_squared_error(y_train, y_train_pred)}')
print(f'MSE test: {mean_squared_error(y_test, y_test_pred)}')


MSE train: 18.579087863398883
MSE test: 17.203890972319577


Теперь предскажем на тестовых данных

In [382]:
test_df = pd.read_csv('test.csv')
test_df.head()

,ID,ApplicationDate,Age,AnnualIncome,CreditScore,LoanAmount,LoanDuration,MaritalStatus,NumberOfDependents,HomeOwnershipStatus,...,JobTenure,EmploymentStatus,EducationLevel,Experience,NetWorth,BaseInterestRate,InterestRate,MonthlyLoanPayment,TotalDebtToIncomeRatio,LoanApproved
0,0,1979-01-03,49,207095,850,14458,60,Single,0,Rent,...,6,Employed,High School,23,10626,0.104458,0.125350,325.532336,0.040650,1
1,1,2006-04-10,28,64653,674,65491,48,Single,2,Own,...,3,Employed,High School,3,299024,0.233491,0.260561,2210.247757,0.454410,0
2,2,2013-11-28,41,300000,768,16811,60,Single,1,Own,...,3,Employed,High School,19,6979,0.147811,0.154571,403.977431,0.034919,1
3,3,1995-08-24,50,300000,850,48737,36,Divorced,0,Mortgage,...,3,Self-Employed,Bachelor,26,8438,0.118737,0.099712,1571.946906,0.085558,1
4,4,1980-06-15,35,300000,672,51916,60,Single,6,Mortgage,...,7,Employed,Associate,12,436216,0.230916,0.237727,1486.675235,0.063707,1


In [388]:
test_df['MaritalStatus'] = test_df['MaritalStatus'].astype('category').cat.codes
test_df['HomeOwnershipStatus'] = test_df['HomeOwnershipStatus'].astype('category').cat.codes
test_df['LoanPurpose'] = test_df['LoanPurpose'].astype('category').cat.codes
test_df['EmploymentStatus'] = test_df['EmploymentStatus'].astype('category').cat.codes
test_df['EducationLevel'] = test_df['EducationLevel'].astype('category').cat.codes

features = ['Age', 'AnnualIncome', 'CreditScore', 'LoanAmount',
       'LoanDuration', 'MaritalStatus', 'NumberOfDependents',
       'HomeOwnershipStatus', 'MonthlyDebtPayments',
       'CreditCardUtilizationRate', 'NumberOfOpenCreditLines',
       'NumberOfCreditInquiries', 'DebtToIncomeRatio', 'BankruptcyHistory',
       'LoanPurpose', 'PreviousLoanDefaults', 'PaymentHistory',
       'LengthOfCreditHistory', 'SavingsAccountBalance',
       'CheckingAccountBalance', 'TotalAssets', 'TotalLiabilities',
       'MonthlyIncome', 'UtilityBillsPaymentHistory', 'JobTenure',
       'EmploymentStatus', 'EducationLevel', 'Experience', 'NetWorth',
       'InterestRate', 'TotalDebtToIncomeRatio', 'LoanApproved']
x = test_df.loc[:, features].values

x = StandardScaler().fit_transform(x)
test_df_stand = pd.DataFrame(data = x, columns = features)

x_test = test_df_stand

y_test_pred = model.predict(x_test)

df_result = pd.DataFrame({
    'ID': test_df['ID'],  
    'RiskScore': y_test_pred  
})

df_result.to_csv('predict.csv', index=False)
